In [1]:
from google.colab import drive
drive.mount('/content/drive/')



Mounted at /content/drive/


In [3]:

import glob

folder = '/content/drive/MyDrive/reddit/cleaned_matched'
folder_loc = glob.glob('{}/*.csv'.format(folder))
folder_loc


['/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_01.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_02.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_03.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_04.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_05.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_06.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_07.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_08.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_09.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_10.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_reddit_submission_2020_11.csv',
 '/content/drive/MyDrive/reddit/cleaned_matched/clean_

In [4]:


import sklearn
import pandas as pd
import numpy as np
#from deep_translator import GoogleTranslator
import xgboost as xgb
import matplotlib.pyplot as plt
import re
#import text2vec

# Create an empty DataFrame with the specified columns

columns = ['Unnamed: 0', 'author', 'id', 'selftext', 'subreddit', 'subreddit_id',
           'title', 'Country', 'selftext_NO_PUNC', 'title_NO_PUNC', 'fentanyl',
           'count_fentanyl', 'heroin', 'count_heroin', 'oxy', 'count_oxy', 'MDMA',
           'count_MDMA', 'INHALANTS', 'count_INHALANTS', 'GHB', 'count_GHB',
           'COCAINE', 'count_COCAINE', 'marijuana', 'count_marijuana',
           'total_flags', 'match', 'total_text_no_punc', 'total_text',
           'concat', 'total_no_drug', 'year']

# Create an empty DataFrame with the specified columns
balanced_sample = pd.DataFrame(columns=columns)
no_included_samples = pd.DataFrame(columns=columns)


def balanced_sample_maker(csv, balanced, left_overs, un_balancer):
    file_path_submission = csv
    submissions = pd.read_csv(file_path_submission)
    submissions['year'] = file_path_submission[-11:-7]


    #submissions = submissions[submissions['no_match'] == 0]
    #submissions['match'] = np.where(submissions['no_match'] == 0, 1,0)

    #submissions['match'] = submissions['no_match']
    submissions[submissions['selftext'] != '[removed]']
    submissions[submissions['selftext'] != '[deleted]']
    submissions['selftext'] = submissions['selftext'].fillna(' ')
    submissions['title'] = submissions['title'].fillna(' ')
    submissions['selftext_NO_PUNC'] = submissions['selftext_NO_PUNC'].fillna(' ')
    submissions['title_NO_PUNC'] = submissions['title_NO_PUNC'].fillna(' ')
    #submissions
    submissions['total_text_no_punc'] = submissions['title_NO_PUNC'] + " " + submissions['selftext_NO_PUNC']
    submissions['total_text'] = submissions['title'] + " " + submissions['selftext']

    submissions = submissions[~submissions['total_text'].str.contains('\[removed\]', case=False)]
    submissions = submissions[~submissions['total_text'].str.contains('\[deleted\]', case=False)]
    removed_pattern = re.compile(r'\[r(e|m)(o|0)v(e|a)ed\]', flags=re.IGNORECASE)
# Filter out rows containing '[removed]' variations in the 'total_text' column
    submissions = submissions[~submissions['total_text'].str.extract(removed_pattern).notna().any(axis=1)]

    lst = submissions.columns.tolist()

    # Splitting column names by commas and flattening the resulting list
    split_columns = [item.split(',') for item in lst]
    flattened_list = [item for sublist in split_columns for item in sublist]

    filtered_list = [item for item in flattened_list[flattened_list.index('title_NO_PUNC') + 1:]]
    filtered_list = [item for item in filtered_list if "count_" not in item]
    filtered_list = [item for item in filtered_list[:filtered_list.index('total_flags'):]]

    submissions['concat'] = submissions[filtered_list].values.tolist()

    submissions['concat'] = submissions['concat'].apply(lambda x: [item for item in x if pd.notna(item)])

    submissions['concat'] = submissions['concat'].apply(lambda x: [item.strip() for sublist in x for item in sublist.split(',')] if x else [])

    def remove_occurrences(row):
        for item in row['concat']:
            # Create a regular expression pattern for whole-word match with case-insensitive flag
            pattern = r'\b{}\b'.format(re.escape(item))
            # Use re.sub to replace occurrences based on the pattern
            row['total_text_no_punc'] = re.sub(pattern, '', row['total_text_no_punc'], flags=re.IGNORECASE)
        return row['total_text_no_punc']

    # Apply the function to each row
    submissions['total_no_drug'] = submissions.apply(remove_occurrences, axis=1)

    def compbl(s):
        return ' '.join(s.split())

    submissions['total_no_drug'] = submissions['total_no_drug'].apply(compbl)


    drug_summary = submissions.columns.tolist()
    count_columns = [col for col in drug_summary if col.startswith('count_')]
    #submissions = submissions[submissions['total_text'].str.contains(re.escape('[removed]'))]
  # there are some files where there is only a single word
    sample_only_drug_word = submissions[submissions['total_no_drug'].isnull()]
    #submissions = submissions[~submissions['total_no_drug'].isnull()]
    submissions = submissions[submissions['total_no_drug'].notna()]

    n = int(round(submissions['match'].sum() * un_balancer))
    sample_no_drug = submissions[submissions['match'] == 0].sample(n)
    not_selected_samples = submissions[~submissions.index.isin(sample_no_drug.index)]
    not_selected_samples = pd.concat([not_selected_samples,sample_only_drug_word])
    #keeping not included samples
    sample_drug = submissions[submissions['match'] == 1]
    balanced = pd.concat([sample_drug,sample_no_drug], ignore_index = True)

    print("total drug related content {}, total length of balanced dataset {}".format(submissions['match'].sum(), len(balanced)))
    return balanced, not_selected_samples





for x in folder_loc:
    bal, ns = balanced_sample_maker(x,
                          balanced_sample,
                         no_included_samples,
                         1.3)
    no_included_samples = pd.concat([no_included_samples,ns], ignore_index = True)
    balanced_sample = pd.concat([balanced_sample, bal], ignore_index = True)



total drug related content 665, total length of balanced dataset 1529


<ipython-input-4-14ca56e13855>:27: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission)


total drug related content 651, total length of balanced dataset 1497
total drug related content 757, total length of balanced dataset 1741
total drug related content 856, total length of balanced dataset 1969


<ipython-input-4-14ca56e13855>:27: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission)


total drug related content 774, total length of balanced dataset 1780


...

total drug related content 707, total length of balanced dataset 1626


<ipython-input-4-14ca56e13855>:27: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission)


total drug related content 645, total length of balanced dataset 1483
total drug related content 683, total length of balanced dataset 1571
total drug related content 723, total length of balanced dataset 1663
total drug related content 769, total length of balanced dataset 1769
total drug related content 813, total length of balanced dataset 1870
total drug related content 959, total length of balanced dataset 2206
total drug related content 855, total length of balanced dataset 1967
total drug related content 819, total length of balanced dataset 1884


<ipython-input-4-14ca56e13855>:27: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  submissions = pd.read_csv(file_path_submission)


total drug related content 965, total length of balanced dataset 2219
total drug related content 957, total length of balanced dataset 2201
total drug related content 933, total length of balanced dataset 2146
total drug related content 825, total length of balanced dataset 1897
total drug related content 838, total length of balanced dataset 1927
total drug related content 887, total length of balanced dataset 2040
total drug related content 808, total length of balanced dataset 1858
total drug related content 857, total length of balanced dataset 1971
total drug related content 753, total length of balanced dataset 1732
total drug related content 1077, total length of balanced dataset 2477
total drug related content 858, total length of balanced dataset 1973
total drug related content 724, total length of balanced dataset 1665
total drug related content 860, total length of balanced dataset 1978
total drug related content 759, total length of balanced dataset 1746
total drug related 

In [7]:
balanced_sample[balanced_sample['total_no_drug'].isnull()]

,Unnamed: 0,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,title_NO_PUNC,...,count_COCAINE,marijuana,count_marijuana,total_flags,match,total_text_no_punc,total_text,concat,total_no_drug,year


In [8]:
no_included_samples.to_csv('/content/drive/MyDrive/reddit/balanced_submission/no_include_sample_sub.csv')


balanced_sample.to_csv('/content/drive/MyDrive/reddit/balanced_submission/balance_sample_sub.csv')



# New Section